## ДЗ 6:
### Экспорт моделей в различные форматы Tensort, ONNX

In [1]:
import ultralytics
from ultralytics import YOLO

In [2]:
import torch
import time
import torch.quantization
from torch import profiler
from torch.profiler import profile, record_function, ProfilerActivity

In [3]:
from utils import get_model_size, profile_model

In [4]:
torch.seed = 42

### Исходная модель

In [5]:
yolo = YOLO('yolov8s.pt')
model = yolo.model
model.fuse()
model.eval()
print()

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv8s summary (fused): 168 layers, 11156544 parameters, 0 gradients, 28.6 GFLOPs


In [7]:
print(profile_model(model))
print(f'model size: {get_model_size(model)}')

STAGE:2023-10-13 20:34:35 7559:3235187 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-10-13 20:34:35 7559:3235187 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-10-13 20:34:35 7559:3235187 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total KFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference       -11.10%  -11729.000us       100.00%     105.668ms     105.668ms           0 b     -24.66 Mb             1            --  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 105.668ms

model size: 42.558837890625


### ONNX

In [11]:
yolo.export(format='onnx')

Ultralytics YOLOv8.0.190 🚀 Python-3.8.0 torch-2.0.1 CPU (Apple M1 Pro)

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)
requirements: Ultralytics requirement ['onnx>=1.12.0'] not found, attempting AutoUpdate...
  Obtaining dependency information for onnx>=1.12.0 from https://files.pythonhosted.org/packages/84/81/59d61d2de3487b2cf04d7e2ada2aa27e02065c07881f7734df61bb3e3a97/onnx-1.14.1-cp38-cp38-macosx_10_12_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:01:01

requirements: AutoUpdate success ✅ 13.6s, installed 1 package: ['onnx>=1.12.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 16.1s, saved as 'yolov8s.onnx' (42.8 MB)

Export complete (20.8s)
Results saved to /Users/timur/Documents/itmo_master/1_term/model_compression/IonovTimur
Predict:     

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



'yolov8s.onnx'

In [28]:
input_example = torch.randn(1, 3, 640, 640)

In [13]:
# pip insall onnxruntime

In [14]:
import onnxruntime as ort
import numpy as np

In [29]:
ort_sess = ort.InferenceSession('yolov8s.onnx')
outputs = ort_sess.run(None, {'images': input_example.float().numpy()})

In [33]:
import time

In [35]:
s = time.time()
ort_sess.run(None, {'images': input_example.float().numpy()})
print(time.time() - s)

0.20946478843688965


In [38]:
s = time.time()
yolo.model(input_example)
print(time.time() - s)

0.7855696678161621
